# **IP3_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.  """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Variables:

1. X_ij = Binary variable that equals 1 if item i is packed into box j and 0 otherwise.
2. Y_j = Binary variable that equals 1 if box j is used and 0 otherwise. 
3. W_i = Weight of item i.
4. C_j = Capacity of box j.
5. B = Upper bound on the amount of boxes available.
6. N = Total number of items.
7. M = Total number of boxes.


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Minimize ∑_(j=1)^M Y_j


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. Each item is packed into one box: ∑_(j=1)^M X_ij = 1, for each i (1 to N)

2. Weight constraint: ∑_(i=1)^N W_i*X_ij ≤ C_j*Y_j, for each j (1 to M)

3. Upper bound on the amount of boxes: ∑_(j=1)^M Y_j ≤ B

4. X_ij, Y_j are binary: X_ij, Y_j ∈ {0,1} for each i (1 to N) and each j (1 to M)


In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [18]:
Markdown(response4.choices[0].message.content)

Let's now implement this model in python pyomo. We will create some random sample data for number of items, number of boxes, weights of the items and capacities of the boxes.

```python
from pyomo.environ import *

# Constant data
N = 10  # number of items
M = 5  # number of boxes
B = 4  # upper limit on available boxes

# Sample data for weights of items and capacities of boxes
W = {i:random.randint(1,20) for i in range(N)}
C = {j:random.randint(50,100) for j in range(M)}

# Model
model = ConcreteModel()

# Variables
model.X = Var(range(N), range(M), within=Binary)
model.Y = Var(range(M), within=Binary)

# Objective
model.obj = Objective(expr=sum(model.Y[j] for j in range(M)), sense=minimize)

# Constraints
model.single_box = ConstraintList()
for i in range(N):
    model.single_box.add(sum(model.X[i,j] for j in range(M)) == 1)

model.weight = ConstraintList()
for j in range(M):
    model.weight.add(sum(W[i]*model.X[i,j] for i in range(N)) <= C[j]*model.Y[j])

model.box_limit = Constraint(expr=sum(model.Y[j] for j in range(M)) <= B)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print('Objective =', model.obj())
```

Sample data is used in this code to stand in for the weights of the items (W) and the capacities of the boxes (C). In a real world scenario, this data would be gathered from the scenario the model is being used for, for example the weight and size of products in a warehouse, the capacity of trucks for transport, etc.

### **Run the code Generated by GPT4**

In [20]:
from pyomo.environ import *
import random
# Constant data
N = 10  # number of items
M = 5  # number of boxes
B = 4  # upper limit on available boxes

# Sample data for weights of items and capacities of boxes
W = {i:random.randint(1,20) for i in range(N)}
C = {j:random.randint(50,100) for j in range(M)}

# Model
model = ConcreteModel()

# Variables
model.X = Var(range(N), range(M), within=Binary)
model.Y = Var(range(M), within=Binary)

# Objective
model.obj = Objective(expr=sum(model.Y[j] for j in range(M)), sense=minimize)

# Constraints
model.single_box = ConstraintList()
for i in range(N):
    model.single_box.add(sum(model.X[i,j] for j in range(M)) == 1)

model.weight = ConstraintList()
for j in range(M):
    model.weight.add(sum(W[i]*model.X[i,j] for i in range(N)) <= C[j]*model.Y[j])

model.box_limit = Constraint(expr=sum(model.Y[j] for j in range(M)) <= B)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print('Objective =', model.obj())




Objective = 2.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [27]:
from pyomo.environ import *

# data inputed by human
W = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
C = 9    # Box capacities
B= 15
N =24 # number of items
M = 15              # Maximum number of used boxes
#end
# Create a simple model
model = ConcreteModel()

# Variables
model.X = Var(range(1,N+1), range(1,M+1), within=Binary)
model.Y = Var(range(1,M+1), within=Binary)

# Objective
model.obj = Objective(expr=sum(model.Y[j] for j in range(1,M+1)), sense=minimize)

# Constraints
model.single_box = ConstraintList()
for i in range(1,N+1):
    model.single_box.add(sum(model.X[i,j] for j in range(1,M+1)) == 1)

model.weight = ConstraintList()
for j in range(1,M+1):
    model.weight.add(sum(W[i]*model.X[i,j] for i in range(1,N+1)) <= C*model.Y[j])

model.box_limit = Constraint(expr=sum(model.Y[j] for j in range(1,M+1)) <= B)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print('Objective =', model.obj())



Objective = 13.0
